In [2]:
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.2 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


- plotlyでのnetwork図描画
  - 散布図でnodeとEdgeを別々にかいて重ね合わせる
  - やりとりの数などを可視化したければ、nodeのサイズやEdgeのサイズの値としてわたせばできそう
- いいかんじの位置にはどうすればできるか?
  - networkxをつかう?
- plotlyではEdgeに情報を持たせることができない
- d3graphならEdgeにデータを持たせられる
  - [plot - Customizing a Networkx graph (or Scatter) with Python Plotly - Stack Overflow](https://stackoverflow.com/questions/50078361/customizing-a-networkx-graph-or-scatter-with-python-plotly)
  - [D3Graph — d3graph d3graph documentation](https://erdogant.github.io/d3graph/pages/html/index.html)
  - [Big Bang network — d3graph d3graph documentation](https://erdogant.github.io/d3graph/pages/html/Examples.html)


- 参考
  - [Pythonでネットワークグラフを描くならNetworkx + Plotlyが便利！ | 子育て×エンジニアの超自由研究ブログ](https://makitat.com/wp-1/2020/python-network-graph-plotly/)
  - [【Python】NetworkX 2.0の基礎的な使い方まとめ - Qiita](https://qiita.com/kzm4269/items/081ff2fdb8a6b0a6112f)

In [3]:
# Create random graph
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(200, 0.125)#networkxのサンプルデータ



In [17]:

# Create Edges
# Add edges as disconnected lines in a single trace and nodes as a scatter trace

#Edgeのデータ作成
edge_x = []
edge_y = []
# networkxのサンプルデータから抜く
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']#この構造はなに?
    x1, y1 = G.nodes[edge[1]]['pos']#この構造はなに?
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)#なぜNone?
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)#なぜNone?

#Edgeのいちを決めて散布図を描く
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),#線のサイズはここで決めてる
    hoverinfo='none',
    mode='lines')#線をひく

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

#Nodeの位置を決めて散布図を描く
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


In [18]:
# Color Node Points
# Color node points by the number of connections.

# Another option would be to size points by the number of connections i.e. node_trace.marker.size = node_adjacencies

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
# node_trace.text = node_text
# Create Network Graph
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

networkxを使う

- やりたいこと
  - from-to dataからつくれるか
- 結論
  - できる
  - ただし、Edgeに情報を持たせることが難しい

In [21]:
import pandas as pd

test_df = pd.DataFrame(
    {
    "from": ["A","A","B","B","C"],
    "to":["B","C","A","C","B"],
    "count":[10,20,10,5,1]
    }
)

In [22]:
test_df

,from,to,count
0,A,B,10
1,A,C,20
2,B,A,10
3,B,C,5
4,C,B,1


In [66]:
G = nx.Graph()


#nodeをつくる
nodes = list(set(pd.concat([test_df["from"], test_df["to"]])))
G.add_nodes_from(nodes)#追加

#edgeをつくる
f_ud_edge = lambda row: tuple(sorted((row["from"], row["to"])))
test_df["graph_edge"] = test_df.apply(f_ud_edge, axis=1)#from,toの組み合わせをつくる
test_df_ndgraph = test_df[["count", "graph_edge"]].groupby("graph_edge").sum()

edges = test_df["graph_edge"].drop_duplicates()
edge_weights = test_df_ndgraph["count"]
G.add_edges_from(edges)


#ポジション決め
pos = nx.spring_layout(G, k=0.3, seed=1)

#nodeの位置
node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

#edgeのいち
edge_x = []
edge_y = []
for edges in G.edges():
    x0, y0 = pos[edge[0]]#node0の位置
    x1, y1 = pos[edge[1]]#node1の位置
    edge_x.append(x0)
    edge_y.append(y0)
    edge_x.append(x1)
    edge_y.append(y1)
    #線を切るために追加する
    edge_x.append(None)
    edge_y.append(None)





In [67]:
pos

{'B': array([0.71411235, 0.74240758]),
 'A': array([-1.        ,  0.24723564]),
 'C': array([ 0.28588765, -0.98964323])}

In [68]:
G.edges()

EdgeView([('B', 'A'), ('B', 'C'), ('A', 'C')])

In [69]:
!pip install d3graph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 820.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 8.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.3 MB/s eta 0:00:00:00:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 kB 5.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.5 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 5.6 MB/s eta 0:00:0000:0100:01
  DEPRECATION: python-louvain is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is 

In [70]:
graph_test_df = test_df.pivot(index="from", columns="to", values="count")

In [71]:
graph_test_df

to,A,B,C
from,,,
A,NaN,10.0,20.0
B,10.0,NaN,5.0
C,NaN,1.0,NaN


In [4]:

from d3graph import d3graph

network = d3graph()
network.graph(graph_test_df)
network.show()

NameError: name 'graph_test_df' is not defined

In [12]:
from d3graph import d3graph
sample = d3graph(collision=2)
adjmat, df = sample.import_example('karate')
sample.graph(adjmat)
sample.show(filepath='/home/ryotani/Documents/work/datamining-chat/slack/d3graph_sample.html')

[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Number of unique nodes: 34
[d3graph] INFO> Slider range is set to [0, 7]
[d3graph] INFO> Write to path: [/home/ryotani/Documents/work/datamining-chat/slack/d3graph_sample.html]
[d3graph] INFO> File already exists and will be overwritten: [/home/ryotani/Documents/work/datamining-chat/slack/d3graph_sample.html]
